In [1]:
import pandas as pd
import os
from tqdm import tqdm
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

#from torchvision import models

import torch
import torchvision
from torchvision import transforms
from torch.autograd import Variable
import torch.nn as nn
from torch import optim
from torch.optim import lr_scheduler
from torchvision.utils import make_grid
from torch.utils.data import Dataset
import torch.functional as F


import time



import itertools
from matplotlib.pyplot import imshow
import matplotlib.cm as cm
import matplotlib.colors as mcolors
import collections
from pprint import pprint

%matplotlib inline

In [2]:
path = r"/blue/eel5840/patricklehman/DLCG_Final"

os.chdir(r'/blue/eel5840/patricklehman/DLCG_Final')

FileNotFoundError: [WinError 3] The system cannot find the path specified: '/blue/eel5840/patricklehman/DLCG_Final'

In [3]:
labels_path = path+r'/onehot_labels.txt'
labels_dframe = pd.read_csv(labels_path)
labels_dframe.columns = ['labels']
print(labels_dframe.head())

FileNotFoundError: [Errno 2] No such file or directory: '/blue/eel5840/patricklehman/DLCG_Final/onehot_labels.txt'

In [4]:
class CelebAFeatures(Dataset):
    def __init__(self, dataframe, directory, transform = None):
        self.df = dataframe
        self.labels = dataframe.labels.values.tolist()
        self.image_names = dataframe.index
        self.directory = directory
        self.transforms = transform
        
    def __len__(self):
        return len(self.df)
    def __getitem__(self, index):
        testpath = self.directory + "/" +self.df.iloc[index,0][0:10]
        image = Image.open(testpath)
        label = self.df.iloc[index,0][10:].replace(' ', '')

        label = torch.FloatTensor([float(item) for item in label])

        sample = {'image': image, 'label': label}
        if self.transforms:
            image = self.transforms(sample['image'])
            sample = {'image': image, 'label': label}
            
        return sample

In [5]:
resize_tensor = transforms.Compose([transforms.Resize((256, 256)),
                           transforms.ToTensor()])


train_dl = CelebAFeatures(labels_dframe[:182398], path + '/data/train_img', transform = resize_tensor)
valid_dl = CelebAFeatures(labels_dframe[182398:], path + '/data/valid_img', transform = resize_tensor)


train_dataloader = torch.utils.data.DataLoader(train_dl, shuffle = False, batch_size = 16, num_workers = 3)
valid_dataloader = torch.utils.data.DataLoader(valid_dl, shuffle = True, batch_size = 16, num_workers = 3)

NameError: name 'labels_dframe' is not defined

In [9]:
class MultiClassifier(nn.Module):
    def __init__(self):
        super(MultiClassifier, self).__init__()
        
        self.conv1 = nn.Sequential(
        nn.Conv2d(3, 64, 3), # 3, 256, 256
        nn.MaxPool2d(2), # op: 16, 127, 127
        nn.ReLU(), # op: 64, 127, 127
        )
        
        self.conv2 = nn.Sequential(
        nn.Conv2d(64, 128, 3), # 64, 127, 127
        nn.MaxPool2d(2), #op: 128, 63, 63
        nn.ReLU() # op: 128, 63, 63
        )
        
        self.conv3 = nn.Sequential(
        nn.Conv2d(128, 256, 3), 
        nn.MaxPool2d(2), 
        
        )
        self.conv4 = nn.Sequential(
        nn.Conv2d(256, 512, 3),
        nn.MaxPool2d(2),
        nn.ReLU(), 
        nn.Dropout(0.2)
        )
        
        self.LL1 = nn.Linear(512 * 14 * 14, 1024)
        self.LL2 = nn.Linear(1024, 256)
        self.LL3 = nn.Linear(256, 40)
        
    def forward(self, t):
        t = self.conv1(t)
        t = self.conv2(t)
        t = self.conv3(t)
        t = self.conv4(t)
        t = t.view(t.size(0), -1)
        t = self.LL1(t)
        t = self.LL2(t)
        t = self.LL3(t)
        output = torch.sigmoid(t)
        return output

In [7]:


def get_accuracy(o_val, pred):
    accuracy = torch.round(pred).eq(o_val).sum().numpy()/len(o_val)
    return accuracy

def fit(epochs, model, dataloader, phase = 'training', volatile = False):
    
  
    
    if phase == 'training':
        model.train()
        
    if phase == 'validataion':
        model.eval()
        volatile = True
        
    running_loss = []
    running_acc = []
    b = 0
    
    for i, data in enumerate(tqdm(dataloader)):
        #removed cuda here
        inputs = data['image']
        target = data['label'] 

        #print(target)
        #inputs, target = torch.Tensor(inputs), torch.Tensor(target)
        try:
            target = torch.FloatTensor(target).cuda()
        except: 
            print("exception occurred")
            print(target)
            
            
        #print(inputs.size())
        inputs, target = Variable(inputs), Variable(target)
        #print("t",target[i])
        if phase == 'training':
            optimizer.zero_grad()
            
        ops = model(inputs.cuda())
        acc_ = []
        for i, d in enumerate(ops, 0):

            acc = get_accuracy(torch.Tensor.cpu(target[i]), torch.Tensor.cpu(d))
            acc_.append(acc)
            loss = criterion(ops, target)

            running_loss.append(loss.item())
            running_acc.append(np.asarray(acc_).mean())
            b += 1
        
        if phase == 'training':
            
            loss.backward()
        
            optimizer.step()
            
    total_batch_loss = np.asarray(running_loss).mean()
    total_batch_acc = np.asarray(running_acc).mean()
    if phase=='training':
        pprint("T_Loss: {}".format(total_batch_loss))
        pprint("T_Acc: {}".format(total_batch_acc))
    if phase=="validation":
        pprint("V_Loss: {}".format(total_batch_loss))
        pprint("V_Acc: {}".format(total_batch_acc))
   
    
    return total_batch_loss, total_batch_acc
def check_cuda():
        _cuda = False
        if torch.cuda.is_available():
            _cuda = True
        return _cuda
    


In [ ]:
is_cuda = check_cuda()
model = MultiClassifier().cuda()
criterion = nn.BCELoss()
optimizer = optim.SGD(model.parameters(), lr = 0.01, momentum = 0.9, weight_decay = 1e-5)
if is_cuda:
    print("On GPU")
    model.cuda()

In [ ]:

dict_to_edit = torch.load('saved_model.pt')
#don't delete till retraining as of 4/18/21
new_model_dict = {}

new_model_dict["conv1.0.weight"] = dict_to_edit.get("ConvLayer1.0.weight")
new_model_dict["conv1.0.bias"] = dict_to_edit.get("ConvLayer1.0.bias")

new_model_dict["conv2.0.weight"] = dict_to_edit.get("ConvLayer2.0.weight")
new_model_dict["conv2.0.bias"] = dict_to_edit.get("ConvLayer2.0.bias")

new_model_dict["conv3.0.weight"] = dict_to_edit.get("ConvLayer3.0.weight")
new_model_dict["conv3.0.bias"] = dict_to_edit.get("ConvLayer3.0.bias")

new_model_dict["conv4.0.weight"] = dict_to_edit.get("ConvLayer4.0.weight")
new_model_dict["conv4.0.bias"] = dict_to_edit.get("ConvLayer4.0.bias")

new_model_dict["LL1.weight"] = dict_to_edit.get("Linear1.weight")
new_model_dict["LL1.bias"] = dict_to_edit.get("Linear1.bias")

new_model_dict["LL2.weight"] = dict_to_edit.get("Linear2.weight")
new_model_dict["LL2.bias"] = dict_to_edit.get("Linear2.bias")

new_model_dict["LL3.weight"] = dict_to_edit.get("Linear3.weight")
new_model_dict["LL3.bias"] = dict_to_edit.get("Linear3.bias")
    


new_model_dict = collections.OrderedDict(new_model_dict)
print(type(new_model_dict))

In [ ]:
model.load_state_dict(torch.load('saved_model.pt'))

num_epochs=4

train_losses = []
train_accuracies = []
val_losses = []
val_accuracies = []

for i in range(1, num_epochs+1):
    pprint("Epoch: {}".format(i))
    train_loss, train_accuracy = fit(i, model, train_dataloader, phase  = 'training')
    val_loss, val_accuracy = fit(i, model, valid_dataloader, phase = 'validation')
    train_losses.append(train_loss)
    train_accuracies.append(train_accuracy)
    val_losses.append(val_loss)
    val_accuracies.append(val_accuracy)
    
torch.save(model.state_dict(), 'saved_model.pt')

In [ ]:
labels = ['5_o_Clock_Shadow','Arched_Eyebrows','Attractive','Bags_Under_Eyes','Bald','Bangs','Big_Lips','Big_Nose','Black_Hair',
'Blond_Hair', 'Blurry','Brown_Hair','Bushy_Eyebrows','Chubby','Double_Chin','Eyeglasses','Goatee','Gray_Hair','Heavy_Makeup',
'High_Cheekbones','Male','Mouth_Slightly_Open','Mustache','Narrow_Eyes','No_Beard','Oval_Face','Pale_Skin','Pointy_Nose',
'Receding_Hairline','Rosy_Cheeks','Sideburns','Smiling','Straight_Hair','Wavy_Hair','Wearing_Earrings','Wearing_Hat',
'Wearing_Lipstick','Wearing_Necklace','Wearing_Necktie','Young']

image_fmt = transforms.Compose([torchvision.transforms.ToPILImage()])

def get_tensor(img):
    tfms = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor()
    ])
    return tfms(Image.open(img)).unsqueeze(0)



def PredictAndGraph(img, label_list, model):
    graphs, (ax1, ax2) = plt.subplots(1,2, gridspec_kw={'width_ratios': [3, 1]}, figsize=(15,5))
    imshowim  = image_fmt(image.squeeze(0))
    ax1.imshow(imshowim)
    ax1.set_xticklabels([])
    ax1.set_yticklabels([])
    output = model(img.cuda())
    output_rounded = torch.round(output)
    output_rounded_numpy = torch.Tensor.cpu(output_rounded).detach().numpy()
    preds = np.where(output_rounded_numpy == 1)[1]
    output_sigfigs = torch.Tensor.cpu(torch.round((output)*100)).detach().numpy()[0]
    output_final = np.argsort(torch.Tensor.cpu(output).detach().numpy())[0][::-1]
    
    label = []
    for i in preds:
        label.append(label_list[i])
        label_estimates = {}
    for i in output_final:
        label_estimates[label_list[int(i)]] = output_sigfigs[int(i)]
        
    info = list(label_estimates.items())[:10]
    
    
    label_names = []
    confidences = []
    for label,confidence in info:
        label_names.append(label.replace("_"," ")+": " + str(round(confidence-.005,2)) + "%")
        confidences.append(confidence)
    label_names.reverse()
    confidences.reverse()
    clist = [ "gray", "indigo", "blue", "cyan", 
             "purple", "red", "orange", "yellow", "green"]
    rvb = mcolors.LinearSegmentedColormap.from_list("", clist)
    ax2.barh(label_names, confidences, color=rvb(np.arange(len(label_names))/len(label_names)))
    ax2.set_xticklabels([])
    ax2.set_title("Top 10 Recognized Features")
    
    #plt.subplots_adjust(wspace=2)
    return 


In [10]:
model = MultiClassifier()
model.load_state_dict(torch.load('saved_model.pt'))
model.eval()
volatile = True

index = np.random.randint(0,16)

#image =  next(itertools.islice(valid_dataloader, index, None))['image'][0]
image = resize_tensor(Image.open(path+'/DaKieran.jpg').rotate(0))

PredictAndGraph(image.unsqueeze(0), labels, model)


RuntimeError: Attempting to deserialize object on a CUDA device but torch.cuda.is_available() is False. If you are running on a CPU-only machine, please use torch.load with map_location=torch.device('cpu') to map your storages to the CPU.